In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '..')
from DataModule.Data_Preparation import CoronnaCERTAINDataset
import evaluate 

import xgboost as xgb
from sklearn.linear_model import LinearRegression

/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
dataset = CoronnaCERTAINDataset(
    library_root='/Users/gaskell/Dropbox/Mac/Desktop/Autoimmune_Disease/Code/ML_RA_EHR/Dataset/',
    challenge="regression",
    dataset='CORRONA CERTAIN',
    process_approach='SC',
    imputation='KNN',
    patient_group='bionaive TNF',
    drug_group='all',
    time_points=(0,3),
    train_test_rate=0.8,
    save_csv=False,
    random_state=2022)

feature engineering, drop columns due to 70% missing value: Index(['smkyrs', 'numcigs', 'rfstatus_impute', 'ccpstatus_impute',
       'statin_use'],
      dtype='object')
save file to: /Users/gaskell/Dropbox/Mac/Desktop/Autoimmune_Disease/Code/ML_RA_EHR/Dataset/.csv_temp/Train.csv
save file to: /Users/gaskell/Dropbox/Mac/Desktop/Autoimmune_Disease/Code/ML_RA_EHR/Dataset/.csv_temp/Test.csv


In [3]:
# read train, test from dataloader
train, train_loc = dataset.get_train()
test, test_loc = dataset.get_test()

In [4]:
# get x,y train and test set
X_train = train.iloc[:,:-1]
y_train = train.iloc[:,-1]
X_test = test.iloc[:,:-1]
y_test = test.iloc[:,-1]

In [5]:
y_train.info()

<class 'pandas.core.series.Series'>
Int64Index: 321 entries, 0 to 345
Series name: DAS28_CRP_3M
Non-Null Count  Dtype  
--------------  -----  
321 non-null    float64
dtypes: float64(1)
memory usage: 5.0 KB


In [6]:
# initialize evaluation module
aml = evaluate.AutoBuild(seed=1, project_name="EHR_RA_SC")
baseline = test['DAS28_CRP_0M']
true = test['DAS28_CRP_3M']

In [7]:
# modelmodule
model = xgb.XGBRegressor()
model.fit(X_train, y_train)
pred = model.predict(X_test)
aml.evaluate("xgb", baseline, true, pred)

/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [9]:
# linear model
model = LinearRegression()
model.fit(X_train, y_train)
pred = model.predict(X_test)
aml.evaluate("lr", baseline, true, pred)

In [10]:
regression, classification = aml.leaderboard()

In [11]:
regression

,model,MAE,MSE,RMSE,R2,Pearson_Correlation
0,xgb,1.051127,1.644979,1.282567,0.164463,0.439333
1,lr,1.036374,1.628608,1.276169,0.172778,0.456856


In [12]:
classification

,model,Accuracy
0,xgb,0.430233
1,lr,0.453488
